In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from IPython.display import display_html
import itertools
import warnings
warnings.filterwarnings("ignore")
%store -r PVALUE_VAR

from Project.Utils.visualize import  search, searchTimeSeries

write_path = os.getcwd() + '/Output' #Path to the folder you want to store the dataframes
output_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'

df = pd.read_csv(output_path + 'GoldDataframe.csv')
corr_df_spearman = pd.read_csv(output_path + 'Corr_DF_pearson.csv', index_col = col_country)



In [2]:
def styler_method(df, name):    
    try:
            styles = [dict(selector="caption", props=[("background-color", "#98D3FF")])]
            left1 = pd.Series([PVALUE_VAR], index=['P-value Spearman'])
            left2 = pd.Series([-1], index=['GDP Spearman Corr'])
            left3 = pd.Series([0], index=['GDP Spearman Corr'])
            dfs = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                        .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                        .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                        .format('{:,.4f}', subset = ['GDP Spearman Corr'])\
                        .format('{:,.12f}', subset = ['P-value Spearman']) \
                        .set_caption(name).set_table_styles(styles)\
                        .set_table_attributes("style='display:inline'")
    except:
        dfs = 'No indicators have been found for the window dataframe in this range.'
        
    return dfs

In [3]:
def timeWindow(Zone, Threshold , Years):    
    df_time = searchTimeSeries(Threshold, Years[0], Years[1], True, 'Country', Zone )
    df_global = searchTimeSeries(Threshold, Years[0], Years[1], False, 'Country', Zone )


    if Years[0] > Years[1]: return print("Please, select a valid range of years.")

    dataframe_name =  str(Years[0]) + '-' + str(Years[1])
    
    space = "\xa0" * 10
    try:
        df_time = styler_method(df_time, dataframe_name)._repr_html_()
    except: 
        df_time = 'No indicators available for the selected parameters'
    
    try:
        df_global = styler_method(df_global, '2000-2020')._repr_html_()
    except: 
        df_global = 'No indicators available for the selected parameters'
    display_html(df_time + space  + df_global, raw=True)
    
intslider = widgets.IntRangeSlider(
    value=[min(set(df['Year'])), max(set(df['Year']))],
    min= min(set(df['Year'])),
    max= max(set(df['Year'])),
    step=1,
    description='Years:',
)

country_drop = widgets.Dropdown(
    options=sorted(set(df['Country'].tolist())),
    value='Afghanistan',
    description='Country:',
)

floatslider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Threshold:',
)

widgets.interact(timeWindow, Zone = country_drop, Threshold = floatslider, Years = intslider)

interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Albania', 'Algeria', 'Angola',…

<function __main__.timeWindow(Zone, Threshold, Years)>

In [4]:
median_corr_df_pearson = pd.merge(corr_df_pearson, region_df, how = 'inner', left_index = True, right_index = True).median().rename('GDP Pearson Corr')
median_corr_df_spearman = pd.merge(corr_df_spearman, region_df, how = 'inner', left_index = True, right_index = True).median().rename('GDP Spearman Corr')

def tableWorldMed(Threshold):
    df = pd.concat([median_corr_df_pearson, median_corr_df_spearman], axis = 1)
    df = df.loc[(abs(df['GDP Pearson Corr']) >= Threshold) & (abs(df['GDP Spearman Corr']) >= Threshold)]

    if df.empty:
        return print("No indicators have been found.")

    df = df.sort_values(by = df.columns[0], ascending = False, key = lambda row: df.sum(axis = 1))

    left1 = pd.Series([-1, -1], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])
    left2 = pd.Series([0, 0], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props = 'color:white; background-color:#929bfc;')\
                 .highlight_between(left = left2, right = 1.5, axis = 1, props = 'color:white; background-color:#b3b9ff;')\
                 .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\

    display(df)

intslider = widgets.IntRangeSlider(
    value=[min(set(df['Year'])), max(set(df['Year']))],
    min= min(set(df['Year'])),
    max= max(set(df['Year'])),
    step=1,
    description='Years:',
)

country_drop = widgets.Dropdown(
    options=sorted(set(df['Country'].tolist())),
    value='Afghanistan',
    description='Country:',
)

floatslider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Threshold:',
)


@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableWorldMed(Threshold)

NameError: name 'corr_df_pearson' is not defined

In [8]:
from math import nan

iterable = list(range(min(set(df['Year'])), max(set(df['Year'])) + 1))
iterable = list(itertools.combinations(iterable, 2))
iterable_aux = iterable.copy()
for years in iterable_aux:
    if (years[1] - years[0]) < 4:
        iterable.remove(years)


indicators = list(df.columns[3:])
indicators.remove('GDP')


df_highest = pd.DataFrame(columns={"Indicator"})
df_highest["Indicator"] = indicators
df_highest["Year range"] = 0
df_highest["Highest positive Spearman corr"] = 0
df_highest["Highest negative Spearman corr"] = 0
df_highest.set_index("Indicator", inplace= True)


for years in iterable:
    df_aux = searchTimeSeries(0, years[0], years[1], True, 'Country', 'Spain' )


    for indicator in indicators:
        indicator_corr_last = df_highest[df_highest.index.get_level_values(0) == indicator]["Highest Spearman corr"][0]

        indicator_corr_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["GDP Spearman Corr"][0]
        indicator_p_value_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["P-value Spearman"][0]
        

        if indicator_corr_aux != nan and indicator_p_value_aux < PVALUE_VAR:
            if abs(indicator_corr_last) < abs(indicator_corr_aux):
                df_highest.at[indicator, "Year range"] = str(years[0]) + '-' + str(years[1])
                df_highest.at[indicator, "Highest Spearman corr"] = indicator_corr_aux

        
display(df_highest)

0 < indicator_corr_aux
-0.9746794344808963 < indicator_corr_aux
-0.9856107606091624 < indicator_corr_aux
-0.991031208965115 < indicator_corr_aux
-0.9999999999999999 < indicator_corr_aux


,Year range,Highest Spearman corr
Indicator,,
AgriShareGDP,2001-2006,-1.000000
CreditToAgriFishForest,2001-2005,0.900000
EmploymentRural,2000-2005,1.000000
% Soldiers,2002-2007,-1.000000
Employment in industry,2001-2006,-1.000000
Employment in services,2001-2006,1.000000
Birth Rate,2000-2004,1.000000
Cost business start-up,2004-2009,-1.000000
Death Rate,2012-2016,-0.900000


In [49]:
df_aux.loc[df_aux['Death Rate': 'P-value Spearman']]

KeyError: 'P-value Spearman'

,Year range,Highest Spearman corr
Indicator,,
AgriShareGDP,0,0
CreditToAgriFishForest,0,0
EmploymentRural,0,0
% Soldiers,0,0
Employment in industry,0,0
Employment in services,0,0
Birth Rate,0,0
Cost business start-up,0,0
Death Rate,0,0
